In [1]:
import numpy as np

In [2]:
from cohlib.conv import (
    conv_z_to_v, 
    conv_v_to_z,
    transform_cov_r2c,
    transform_cov_c2r,
    rearrange_mat,
    reverse_rearrange_mat,
)
def quick_conv_c2r(ccov, K):
    return reverse_rearrange_mat(transform_cov_c2r(ccov), K)
def quick_conv_r2c(rcov, K):
    return transform_cov_r2c(rearrange_mat(rcov, K))

def reorder_ve2vri(v_vec_elementwise):
    """Convert [Re(z1), Im(z1), Re(z2), Im(z2)] to 
        [Re(z1), Re(z2), Im(z1), Im(z2)]
    """
    v_temp = v.reshape(-1,2)
    v_ri = np.concatenate([v_temp[:,0], v_temp[:,1]])
    return v_ri

def reorder_vri2ve(v_ri):
    v_e = v_ri.reshape(2,-1).flatten(order='F')
    return v_e

### real / complex vector conversion

In [3]:
K = 3
# first let's look at z -> v
z = np.random.randn(K) + 1j*np.random.rand(K)
v = conv_z_to_v(z, axis=0, dc=False)
z2 = conv_v_to_z(v, axis=0, dc=False)
np.isclose(z,z2)

array([ True,  True,  True])

In [4]:
# next, v -> z 
v = np.random.randn(K*2) 
z = conv_v_to_z(v, axis=0, dc=False)
v2 = conv_z_to_v(z, axis=0, dc=False)
np.isclose(v,v2)

array([ True,  True,  True,  True,  True,  True])

In [5]:
print(z)
print(v)

[-0.46003211+0.11975686j -0.50021042+0.21888811j  0.14952563+0.20228625j]
[-0.46003211  0.11975686 -0.50021042  0.21888811  0.14952563  0.20228625]


In [6]:
v_ri = reorder_ve2vri(v)
v_ri
print(z)
print(v_ri)

[-0.46003211+0.11975686j -0.50021042+0.21888811j  0.14952563+0.20228625j]
[-0.46003211 -0.50021042  0.14952563  0.11975686  0.21888811  0.20228625]


In [7]:
v_ri = reorder_ve2vri(v)
v_e = reorder_vri2ve(v_ri)

In [8]:
print(v)
print(v_e)

[-0.46003211  0.11975686 -0.50021042  0.21888811  0.14952563  0.20228625]
[-0.46003211  0.11975686 -0.50021042  0.21888811  0.14952563  0.20228625]


### z -> v outer product

In [9]:
# next 
z = np.random.randn(K) + 1j*np.random.rand(K)
zzT = np.outer(z, z.conj())
v = conv_z_to_v(z, axis=0, dc=False)
vvT = np.outer(v, v)
vvT_complex = quick_conv_r2c(vvT, K)

print(np.round(zzT,3))
print(np.round(vvT_complex, 3))
print(np.isclose(zzT, vvT_complex))

[[ 2.854+0.j     3.519-0.161j -0.015+0.451j]
 [ 3.519+0.161j  4.348-0.j    -0.043+0.555j]
 [-0.015-0.451j -0.043-0.555j  0.071-0.j   ]]
[[ 2.854+0.j     3.519-0.161j -0.015+0.451j]
 [ 3.519+0.161j  4.348+0.j    -0.043+0.555j]
 [-0.015-0.451j -0.043-0.555j  0.071+0.j   ]]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


In [10]:
print(np.round(z, 4))
print(np.round(z.conj(), 4))

[-1.5774+0.6047j -1.979 +0.6565j  0.1036+0.246j ]
[-1.5774-0.6047j -1.979 -0.6565j  0.1036-0.246j ]


In [11]:
z * z.conj()

array([2.85375101+7.50930680e-18j, 4.34762622-6.85936565e-17j,
       0.07125722-1.14523019e-18j])

In [12]:
np.outer(z,z.conj())

array([[ 2.85375101+7.50930680e-18j,  3.51867538-1.61140611e-01j,
        -0.01457236+4.50708345e-01j],
       [ 3.51867538+1.61140611e-01j,  4.34762622-6.85936565e-17j,
        -0.04341753+5.54900603e-01j],
       [-0.01457236-4.50708345e-01j, -0.04341753-5.54900603e-01j,
         0.07125722-1.14523019e-18j]])

### v -> z

In [48]:
K = 2
Kv = int(2*K)
v = np.random.randn(Kv)
vvT = np.outer(v, v)
z = conv_v_to_z(v, axis=0, dc=False)
zzT = np.outer(z, z.conj())
zzT_real = transform_cov_c2r(zzT)

print(np.round(vvT, 3))
print(np.round(zzT_real,3))
print(np.isclose(vvT, zzT_real))

[[ 1.151 -0.936  1.221 -2.211]
 [-0.936  0.761 -0.993  1.797]
 [ 1.221 -0.993  1.295 -2.345]
 [-2.211  1.797 -2.345  4.245]]
[[ 0.956  1.509  0.    -0.609]
 [ 1.509  2.77   0.609  0.   ]
 [-0.     0.609  0.956  1.509]
 [-0.609 -0.     1.509  2.77 ]]
[[False False False False]
 [False False False False]
 [False False False False]
 [False False False False]]


In [49]:
# next 
v_ri = reorder_ve2vri(v)
vvT = np.outer(v_ri, v_ri)
z = conv_v_to_z(v, axis=0, dc=False)
zzT = np.outer(z, z.conj())
zzT_real = transform_cov_c2r(zzT)
# zzT_real2complex = transform_cov_c2r(zzT_real, 2)
# zzT_real2complex2real = quick_conv_c2r(zzT_real2complex, 2)

print(np.round(vvT, 3))
print(np.round(zzT_real,3))
print(np.isclose(vvT, zzT_real))

[[ 1.151  1.221 -0.936 -2.211]
 [ 1.221  1.295 -0.993 -2.345]
 [-0.936 -0.993  0.761  1.797]
 [-2.211 -2.345  1.797  4.245]]
[[ 0.956  1.509  0.    -0.609]
 [ 1.509  2.77   0.609  0.   ]
 [-0.     0.609  0.956  1.509]
 [-0.609 -0.     1.509  2.77 ]]
[[False False False False]
 [False False False False]
 [False False False False]
 [False False False False]]


In [50]:
# next 
v_ri = reorder_ve2vri(v)
vvT = np.outer(v_ri, v_ri)
vvT_block = transform_cov_c2r(transform_cov_r2c(vvT))
z = conv_v_to_z(v, axis=0, dc=False)
zzT = np.outer(z, z.conj())
zzT_real = transform_cov_c2r(zzT)

print(np.round(vvT_block, 3))
print(np.round(zzT_real,3))
print(np.isclose(vvT_block, zzT_real))

[[ 0.956  1.509 -0.    -0.609]
 [ 1.509  2.77   0.609 -0.   ]
 [ 0.     0.609  0.956  1.509]
 [-0.609  0.     1.509  2.77 ]]
[[ 0.956  1.509  0.    -0.609]
 [ 1.509  2.77   0.609  0.   ]
 [-0.     0.609  0.956  1.509]
 [-0.609 -0.     1.509  2.77 ]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [45]:
K

2

In [53]:
vvT = np.outer(v, v)
z = conv_v_to_z(v, axis=0, dc=False)
zzT = np.outer(z, z.conj())
zzT_real = reverse_rearrange_mat(transform_cov_c2r(zzT), K)
# zzT_real_mod = transform_cov_c2r(transform_cov_r2c(zzT_real))

print(np.round(vvT, 3))
print(np.round(zzT_real,3))
print(np.isclose(vvT, zzT_real))

# print(np.round(zzT_real_mod,3))
# print(np.isclose(vvT, zzT_real_mod))

[[ 1.151 -0.936  1.221 -2.211]
 [-0.936  0.761 -0.993  1.797]
 [ 1.221 -0.993  1.295 -2.345]
 [-2.211  1.797 -2.345  4.245]]
[[ 0.956  0.     1.509 -0.609]
 [-0.     0.956  0.609  1.509]
 [ 1.509  0.609  2.77   0.   ]
 [-0.609  1.509 -0.     2.77 ]]
[[False False False False]
 [False False False False]
 [False False False False]
 [False False False False]]


### z -> v

In [16]:
v = np.random.randn(4)
vvT = np.outer(v, v)
z = conv_v_to_z(v, axis=0, dc=False)
zzT = np.outer(z, z.conj())
zzT_real = transform_cov_c2r(zzT)

print(np.round(vvT, 3))
print(np.round(zzT_real,3))
print(np.isclose(vvT, zzT_real)) 

[[ 0.656 -0.264 -0.37  -0.194]
 [-0.264  0.106  0.149  0.078]
 [-0.37   0.149  0.209  0.109]
 [-0.194  0.078  0.109  0.057]]
[[ 0.381 -0.146 -0.    -0.171]
 [-0.146  0.133  0.171  0.   ]
 [ 0.     0.171  0.381 -0.146]
 [-0.171 -0.    -0.146  0.133]]
[[False False False False]
 [False False False False]
 [False False False False]
 [False False False False]]


## Next let's test if
$\text{conv}(\sum_i \left[\mathbf{A}_i^r + \mathbf{B}_i^r \right]) =
\sum_i \text{conv}(\mathbf{A}_i^r + \mathbf{B}_i^r )$

### first we'll test 
$\text{conv}(\sum_i \mathbf{A}_i^r ) =
\sum_i \text{conv}(\mathbf{A}_i^r )$

In [104]:
K = 4
L = 5
zs = np.stack([np.random.randn(K) + 1j*np.random.rand(K) for l in range(L)])
zzTs_sum = np.zeros((K,K), dtype=complex)
for l in range(L):
    zzT = np.outer(zs[l,:], zs[l,:].conj())
    zzTs_sum += zzT


vs = conv_z_to_v(zs, axis=1, dc=False)
vvTs_sum = np.zeros((2*K,2*K))
for l in range(L):
    vvT = np.outer(vs[l,:], vs[l,:])
    vvTs_sum += vvT

In [105]:
zzT_l = np.round(np.outer(zs[l,:], zs[l,:].conj()),4)
print(zzT_l)
vvT_l = np.round(np.outer(vs[l,:], vs[l,:]),4)
print(vvT_l)
rearr_vvT_l = rearrange_mat(vvT_l, K)
print(rearr_vvT_l)
complex_rearr_vvT_l = transform_cov_r2c(rearrange_mat(vvT_l, K))
print(complex_rearr_vvT_l)
print(np.isclose(zzT_l, complex_rearr_vvT_l))

[[ 0.5631+0.j      0.4231-0.2678j  0.0773+0.37j   -0.0309+0.9285j]
 [ 0.4231+0.2678j  0.4453+0.j     -0.1179+0.3148j -0.4649+0.6829j]
 [ 0.0773-0.37j   -0.1179-0.3148j  0.2538-0.j      0.6059+0.1478j]
 [-0.0309-0.9285j -0.4649-0.6829j  0.6059-0.1478j  1.5328-0.j    ]]
[[ 0.0418 -0.1475  0.1016 -0.091  -0.0912 -0.0477 -0.2456 -0.0608]
 [-0.1475  0.5213 -0.3588  0.3215  0.3223  0.1686  0.8677  0.2147]
 [ 0.1016 -0.3588  0.247  -0.2213 -0.2219 -0.116  -0.5973 -0.1478]
 [-0.091   0.3215 -0.2213  0.1983  0.1988  0.104   0.5351  0.1324]
 [-0.0912  0.3223 -0.2219  0.1988  0.1993  0.1042  0.5365  0.1328]
 [-0.0477  0.1686 -0.116   0.104   0.1042  0.0545  0.2806  0.0694]
 [-0.2456  0.8677 -0.5973  0.5351  0.5365  0.2806  1.4444  0.3574]
 [-0.0608  0.2147 -0.1478  0.1324  0.1328  0.0694  0.3574  0.0884]]
[[ 0.0418  0.1016 -0.0912 -0.2456 -0.1475 -0.091  -0.0477 -0.0608]
 [ 0.1016  0.247  -0.2219 -0.5973 -0.3588 -0.2213 -0.116  -0.1478]
 [-0.0912 -0.2219  0.1993  0.5365  0.3223  0.1988  0.1042  0

In [106]:
complex_rearr_vvTs_sum = transform_cov_r2c(rearrange_mat(vvTs_sum, K))

In [107]:
print(np.round(zzTs_sum,4))
print(np.round(complex_rearr_vvTs_sum,4))
print(np.isclose(zzTs_sum, complex_rearr_vvTs_sum))

[[ 7.3093+0.j      5.908 +0.309j  -2.3616+0.597j   2.3085+0.8085j]
 [ 5.908 -0.309j   9.4653-0.j     -2.8232-2.8508j  3.7932-0.933j ]
 [-2.3616-0.597j  -2.8232+2.8508j  8.4227-0.j     -1.8585+1.5771j]
 [ 2.3085-0.8085j  3.7932+0.933j  -1.8585-1.5771j  6.7152+0.j    ]]
[[ 7.3093+0.j      5.908 +0.309j  -2.3616+0.597j   2.3085+0.8085j]
 [ 5.908 -0.309j   9.4653+0.j     -2.8232-2.8508j  3.7932-0.933j ]
 [-2.3616-0.597j  -2.8232+2.8508j  8.4227+0.j     -1.8585+1.5771j]
 [ 2.3085-0.8085j  3.7932+0.933j  -1.8585-1.5771j  6.7152+0.j    ]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


So, we have confirmed
$\text{conv}(\sum_i \mathbf{A}_i^r ) =
\sum_i \text{conv}(\mathbf{A}_i^r )$

### Now for
$\text{conv}(\sum_i \left[\mathbf{A}_i^r + \mathbf{B}_i^r \right]) =
\sum_i \text{conv}(\mathbf{A}_i^r + \mathbf{B}_i^r )$

In [108]:
L = 5
z1s = np.stack([np.random.randn(K) + 1j*np.random.rand(K) for l in range(L)])
z2s = np.stack([np.random.randn(K) + 1j*np.random.rand(K) for l in range(L)])
zzTs_12sum = np.zeros((K,K), dtype=complex)
for l in range(L):
    zz1T = np.outer(z1s[l,:], z1s[l,:].conj())
    zz2T = np.outer(z2s[l,:], z2s[l,:].conj())
    zzTs_12sum += (zz1T + zz2T)


v1s = conv_z_to_v(z1s, axis=1, dc=False)
v2s = conv_z_to_v(z2s, axis=1, dc=False)
vvTs_12sum = np.zeros((2*K,2*K))
for l in range(L):
    vv1T = np.outer(v1s[l,:], v1s[l,:])
    vv2T = np.outer(v2s[l,:], v2s[l,:])
    vvTs_12sum += (vv1T + vv2T)

In [119]:
l = 4
zz1T_l = np.outer(z1s[l,:], z2s[l,:].conj())
zz2T_l = np.outer(z2s[l,:], z2s[l,:].conj())
zzT_12sum_l = zz1T_l + zz2T_l
print(np.round(zzT_12sum_l,1))
vv1T_l = np.outer(v1s[l,:], v2s[l,:].conj())
vv2T_l = np.outer(v2s[l,:], v2s[l,:].conj())
vvT_12sum_l = vv1T_l + vv2T_l
print(np.round(vvT_12sum_l,1))
rearr_vvT_12sum_l = rearrange_mat(vvT_12sum_l, K)
print(np.round(rearr_vvT_12sum_l, 1))
complex_rearr_vvT_12sum_l = transform_cov_r2c(rearrange_mat(vvT_12sum_l, K))
print(np.round(complex_rearr_vvT_12sum_l,1))
print(np.isclose(zzT_12sum_l, complex_rearr_vvT_12sum_l))

[[ 3.1-0.3j -0.5+0.6j -2.6+0.8j  0.9+0.6j]
 [-0.9-0.6j  0.3-0.j   0.8+0.4j -0.1-0.4j]
 [-1.2-0.3j  0.3-0.2j  1.1+0.1j -0.3-0.4j]
 [ 0.2-1.7j  0.3+0.3j  0.2+1.5j  0.4-0.4j]]
[[ 2.9 -0.6 -0.6 -0.4 -2.6 -0.   0.7 -0.8]
 [-0.9  0.2  0.2  0.1  0.8  0.  -0.2  0.2]
 [-0.9  0.2  0.2  0.1  0.8  0.  -0.2  0.3]
 [-0.4  0.1  0.1  0.1  0.4  0.  -0.1  0.1]
 [-1.3  0.2  0.3  0.2  1.1  0.  -0.3  0.4]
 [-0.1  0.   0.   0.   0.1  0.  -0.   0. ]
 [-0.2  0.   0.   0.   0.1  0.  -0.   0. ]
 [-1.7  0.3  0.3  0.2  1.5  0.  -0.4  0.5]]
[[ 2.9 -0.6 -2.6  0.7 -0.6 -0.4 -0.  -0.8]
 [-0.9  0.2  0.8 -0.2  0.2  0.1  0.   0.3]
 [-1.3  0.3  1.1 -0.3  0.2  0.2  0.   0.4]
 [-0.2  0.   0.1 -0.   0.   0.   0.   0. ]
 [-0.9  0.2  0.8 -0.2  0.2  0.1  0.   0.2]
 [-0.4  0.1  0.4 -0.1  0.1  0.1  0.   0.1]
 [-0.1  0.   0.1 -0.   0.   0.   0.   0. ]
 [-1.7  0.3  1.5 -0.4  0.3  0.2  0.   0.5]]
[[ 3.1-0.3j -0.5+0.6j -2.6+0.8j  0.9+0.6j]
 [-0.9-0.6j  0.3-0.j   0.8+0.4j -0.1-0.4j]
 [-1.2-0.3j  0.3-0.2j  1.1+0.1j -0.3-0.4j]
 [ 0.2-1

In [110]:
complex_rearr_vvTs_12sum = transform_cov_r2c(rearrange_mat(vvTs_12sum, K))

In [111]:
print(np.round(zzTs_12sum,4))
print(np.round(complex_rearr_vvTs_12sum,4))
print(np.isclose(zzTs_12sum, complex_rearr_vvTs_12sum))

[[15.6993-0.j     -0.7628+2.2953j -3.0084+3.8642j -0.7134+3.0341j]
 [-0.7628-2.2953j 12.9385-0.j      0.728 +1.5437j  7.7904-0.9607j]
 [-3.0084-3.8642j  0.728 -1.5437j  8.1513+0.j      2.4972-0.553j ]
 [-0.7134-3.0341j  7.7904+0.9607j  2.4972+0.553j   8.9301+0.j    ]]
[[15.6993+0.j     -0.7628+2.2953j -3.0084+3.8642j -0.7134+3.0341j]
 [-0.7628-2.2953j 12.9385+0.j      0.728 +1.5437j  7.7904-0.9607j]
 [-3.0084-3.8642j  0.728 -1.5437j  8.1513+0.j      2.4972-0.553j ]
 [-0.7134-3.0341j  7.7904+0.9607j  2.4972+0.553j   8.9301+0.j    ]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
